# XGBoost (eXtreme Gradient Boosting)

XGBoost, GBM'in hız ve tahmin performansını arttırmak üzere optimize edilmiş; ölçeklenebilir ve farklı platformlara entegre edilebilir versiyonudur.

In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

warnings.simplefilter(action='ignore', category=Warning)

from xgboost import XGBClassifier
# !pip install lightgbm
# from lightgbm import LGBMClassifier
# !pip install catboost
# from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv("datasets/diabetes.csv")

In [3]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

## XGBoost

In [4]:
xgboost_model = XGBClassifier(random_state=17)

Hiperparametre optimizasyonu yapmadan önceki hatalarımızı hesaplayalım.

In [5]:
cv_results = cross_validate(xgboost_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print("Accuracy: ", cv_results["test_accuracy"].mean())
print("F1: ", cv_results["test_f1"].mean())
print("ROC-AUC: ", cv_results["test_roc_auc"].mean())

Accuracy:  0.7526525761819879
F1:  0.6317893713482235
ROC-AUC:  0.7987134870719776


Şimdi test etmek istediğimiz parametrelerin değerlerini tutan bir sözlük yapısı oluşturmamız lazım. Önce xgboost modelinin parametrelerine bakıp sonra bu sözlük yapısını oluşturalım.

In [6]:
xgboost_model.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': False,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': 17,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [8]:
xgboost_params = {"learning_rate": [0.1, 0.01, 0.001],
                 "max_depth": [5, 8, None],
                 "n_estimators": [100, 500, 1000],
                 "colsample_bytree": [None, 0.5, 0.7, 1]} #colsample_bytree, subsample ile aynı parametre, ismi farklı

In [9]:
xgboost_best_grid = GridSearchCV(xgboost_model, xgboost_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  4.1min finished


In [11]:
xgboost_best_grid.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.001,
 'max_depth': 5,
 'n_estimators': 500}

Modelimiz için en verimli parametreleri bulduk. Şimdi bu parametreler ile final modelimizi kuralım ve kurduğumuz modelin hata değerlerini hesaplayıp önceki model ile karşılaştıralım.

In [13]:
xgboost_final = xgboost_model.set_params(**xgboost_best_grid.best_params_, random_state=17).fit(X, y)

In [15]:
cv_results = cross_validate(xgboost_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print("Accuracy: ", cv_results["test_accuracy"].mean())
print("F1: ", cv_results["test_f1"].mean())
print("ROC-AUC: ", cv_results["test_roc_auc"].mean())

Accuracy:  0.7669892199303965
F1:  0.607939472809153
ROC-AUC:  0.8253396226415095
